# do a transitive query of a wikidata concept and search CU Experts elastic endpoint for all people who have this concept in their elastic document

### Use anaconda shell to add dependencies. eg: conda install -c conda-forge sparqlwrapper

In [18]:
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import namedtuple
import json
import requests

endpoint_url = "https://query.wikidata.org/sparql"
ES_person_endpoint_url = "https://experts.colorado.edu/es/fis/person/_search?size=5000"
ES_pub_endpoint_url = "https://experts.colorado.edu/es/fis/person/_search"
headers = {'Content-type': 'application/json'}
search_term = "artificial intelligence"
person_termlist = dict()

# Find the wikidata entity where the wikidata entity label matches the search_term string AND it's a concept - Q151885
# Then find all subclasses of that concept term in wikidata
query = """SELECT ?subitem ?subitemLabel
WHERE {

  ?item ?label \"""" + search_term + """\"@en.
    ?item wdt:P279* wd:Q11862829.
  ?subitem wdt:P279+ ?item.
  SERVICE wikibase:label { bd:serviceParam wikibase:language \"en\". }
  } """



In [19]:
def WdataSearch(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


In [20]:
def ESsearch(uri, term):
    """Simple Elasticsearch Query"""
    query = json.dumps({
        "query": {
#            "match": {
#                "researchArea.name.exact": term
#            }
             "query_string": { "query": term }
        }
    })
    response = requests.get(uri, headers=headers, data=query)
    results = json.loads(response.text)
    return results



In [22]:
print ("Only displaying results with a search score > .5")
print (search_term)
ESresults = ESsearch(ES_person_endpoint_url,search_term)
for eresult in ESresults['hits']['hits']:
      if eresult["_score"] >= 4:
        #person=personterms(search_term,eresult["_score"])
        #person.name = eresult["_source"]["name"]
        termlist=[]
        termscore={search_term,eresult["_score"]}
        termlist.append(termscore)
        person_termlist[eresult["_source"]["name"]]=termlist
        print (eresult["_score"], eresult["_source"]["name"], eresult["_source"]["uri"])

Wresults = WdataSearch(endpoint_url, query)

print (query)

for result in Wresults["results"]["bindings"]:
   print ("\n")
   print (result['subitemLabel']['value'], result['subitem']['value'])
   ESresults = ESsearch(ES_person_endpoint_url,result['subitemLabel']['value'])
   print ("Total people who match in CU Experts: ", ESresults['hits']['total'])
   for eresult in ESresults['hits']['hits']:
      if eresult["_score"] >= 4:
        print (eresult["_score"], eresult["_source"]["name"], eresult["_source"]["uri"])
        termscore={result['subitemLabel']['value'],eresult["_score"]}
        if eresult["_source"]["name"] in person_termlist:
            person_termlist[eresult["_source"]["name"]].append(termscore)
        else:
            termlist=[]
            termlist.append(termscore)
            person_termlist[eresult["_source"]["name"]]=termlist

print ("\nList of persons and thier terms")
for result in person_termlist:
    print(result,": ")
    for terms in person_termlist[result]:
       print("\t", terms)


Only displaying results with a search score > .5
artificial intelligence
12.571557 Bradley, Liz https://experts.colorado.edu/individual/fisid_100546
12.279214 Burke, Robin D. https://experts.colorado.edu/individual/fisid_165005
11.894519 Roncone, Alessandro https://experts.colorado.edu/individual/fisid_164509
10.388216 Stoica, Maria https://experts.colorado.edu/individual/fisid_157419
10.100544 Hulden, Mans https://experts.colorado.edu/individual/fisid_154602
9.7551775 Surden, Harry https://experts.colorado.edu/individual/fisid_146083
9.209103 Yeh, Tom https://experts.colorado.edu/individual/fisid_151584
8.954992 Martin, James H https://experts.colorado.edu/individual/fisid_100495
8.644597 Tan, Chenhao https://experts.colorado.edu/individual/fisid_158938
8.587057 Ward, Wayne Hinson https://experts.colorado.edu/individual/fisid_114680
8.198035 Sumner, Tamara https://experts.colorado.edu/individual/fisid_105742
7.3356714 Palmer, Martha Stone https://experts.colorado.edu/individual/fisid_

Total people who match in CU Experts:  62
6.147041 McDonald, Robert H https://experts.colorado.edu/individual/fisid_163978
5.6116705 Lowry, Christopher  A. https://experts.colorado.edu/individual/fisid_143371
5.35221 Tsai, Pei-San https://experts.colorado.edu/individual/fisid_115292
5.1933613 Bustamante, Heidi Margarita https://experts.colorado.edu/individual/fisid_146491
5.1906743 Casagrand, Janet L https://experts.colorado.edu/individual/fisid_100934
5.028276 Hertzberg, Jean R https://experts.colorado.edu/individual/fisid_105315
4.9344025 Finan, Donald S. https://experts.colorado.edu/individual/fisid_122670
4.9344025 Enoka, Roger M https://experts.colorado.edu/individual/fisid_110122
4.744708 Arehart, Kathryn H https://experts.colorado.edu/individual/fisid_105561
4.744708 Morton, Yu T https://experts.colorado.edu/individual/fisid_159076
4.741744 Wright, Kenneth P https://experts.colorado.edu/individual/fisid_125586
4.734184 Watkins, Linda R https://experts.colorado.edu/individual/fis

Total people who match in CU Experts:  53
6.951008 Mayer, Stephanie S https://experts.colorado.edu/individual/fisid_114948
5.245284 Medeiros, Daniel Meulemans https://experts.colorado.edu/individual/fisid_145697
5.1846714 Copley, Shelley https://experts.colorado.edu/individual/fisid_104067
5.0827518 Kociolek, John Patrick https://experts.colorado.edu/individual/fisid_145728
5.0345297 Adams, William  Walter, III https://experts.colorado.edu/individual/fisid_103612
5.0345297 Davies, Kendi F https://experts.colorado.edu/individual/fisid_142304
5.0345297 Schmidt, Steve https://experts.colorado.edu/individual/fisid_103713
4.9360037 Watson, Karli K https://experts.colorado.edu/individual/fisid_155058
4.809784 Johnson, Pieter TJ https://experts.colorado.edu/individual/fisid_143590
4.5327935 Hirsh, Aaron Elliot https://experts.colorado.edu/individual/fisid_146025
4.5327935 Demmig-Adams, Barbara https://experts.colorado.edu/individual/fisid_105649
4.5327935 Song, Se Jin https://experts.colorado

Total people who match in CU Experts:  1004
14.909628 Stoica, Maria https://experts.colorado.edu/individual/fisid_157419
13.438344 Jones, Matt https://experts.colorado.edu/individual/fisid_144611
11.372292 Buttenfield, Barbara P https://experts.colorado.edu/individual/fisid_107860
11.0800905 Surden, Harry https://experts.colorado.edu/individual/fisid_146083
9.265396 O'Reilly, Randall Charles https://experts.colorado.edu/individual/fisid_110512
9.149506 Munakata, Yuko https://experts.colorado.edu/individual/fisid_125036
8.436631 Hayes, Bradley Hamilton https://experts.colorado.edu/individual/fisid_159810
7.661974 Paul, Michael J https://experts.colorado.edu/individual/fisid_156070
5.5000443 Keegan, Brian https://experts.colorado.edu/individual/fisid_158122
4.932557 Lv, Qin (Christine) https://experts.colorado.edu/individual/fisid_145832
4.65767 Elsborg, Don https://experts.colorado.edu/individual/fisid_145427
4.2406573 Goldstein, Bruce Evan https://experts.colorado.edu/individual/fisid_

Total people who match in CU Experts:  260
12.571557 Bradley, Liz https://experts.colorado.edu/individual/fisid_100546
12.279214 Burke, Robin D. https://experts.colorado.edu/individual/fisid_165005
11.894519 Roncone, Alessandro https://experts.colorado.edu/individual/fisid_164509
10.388216 Stoica, Maria https://experts.colorado.edu/individual/fisid_157419
10.100544 Hulden, Mans https://experts.colorado.edu/individual/fisid_154602
9.7551775 Surden, Harry https://experts.colorado.edu/individual/fisid_146083
9.209103 Yeh, Tom https://experts.colorado.edu/individual/fisid_151584
8.954992 Martin, James H https://experts.colorado.edu/individual/fisid_100495
8.644597 Tan, Chenhao https://experts.colorado.edu/individual/fisid_158938
8.587057 Ward, Wayne Hinson https://experts.colorado.edu/individual/fisid_114680
8.198035 Sumner, Tamara https://experts.colorado.edu/individual/fisid_105742
7.3356714 Palmer, Martha Stone https://experts.colorado.edu/individual/fisid_138162
7.3356714 Mathys, Peter

...

	 {'Marketing decision support system', 6.4134583}
	 {'Marketing decision support system', 6.4134583}
Echchaibi, Nabil : 
	 {0.9558002, 'knowledge representation and reasoning'}
	 {'Cognitive science of religion', 3.7330039}
	 {'cognitive anthropology', 2.654962}
Stade, Eric : 
	 {0.9558002, 'knowledge representation and reasoning'}
	 {0.8841809, 'part-of-speech tagging'}
	 {4.1467443, 'Morphological analysis'}
	 {0.8841809, 'Cognitive science of religion'}
	 {'independent component analysis', 4.1467443}
	 {'archetypal analysis', 4.1467443}
	 {4.1467443, 'Morphological analysis'}
	 {'independent component analysis', 4.1467443}
	 {'archetypal analysis', 4.1467443}
Pieplow, Nathan D : 
	 {0.94662213, 'knowledge representation and reasoning'}
Su, Tin Tin : 
	 {0.94132936, 'knowledge representation and reasoning'}
	 {1.6345303, 'part-of-speech tagging'}
	 {1.6345303, 'Cognitive science of religion'}
	 {'developmental cognitive neuroscience', 3.586223}
Hynek, Brian Michael : 
	 {0.9297767, 

	 {'machine learning in bioinformatics', 2.2414293}
Lotoaniu, Tapuosi M : 
	 {2.2414293, 'artificial intelligence in fiction'}
	 {2.2414293, 'Artificial intelligence in Wikimedia projects'}
	 {'machine learning in bioinformatics', 2.2414293}
Kofler, Jonathan D : 
	 {2.2414293, 'artificial intelligence in fiction'}
	 {2.2414293, 'Artificial intelligence in Wikimedia projects'}
	 {'machine learning in bioinformatics', 2.2414293}
Aikin, Kenneth C : 
	 {2.2414293, 'artificial intelligence in fiction'}
	 {2.2414293, 'Artificial intelligence in Wikimedia projects'}
	 {'machine learning in bioinformatics', 2.2414293}
Matrosov, Sergey Y : 
	 {2.2414293, 'artificial intelligence in fiction'}
	 {2.2414293, 'Artificial intelligence in Wikimedia projects'}
	 {'machine learning in bioinformatics', 2.2414293}
Wolter, Sonja E : 
	 {2.2414293, 'artificial intelligence in fiction'}
	 {2.2414293, 'Artificial intelligence in Wikimedia projects'}
	 {'machine learning in bioinformatics', 2.2414293}
Scheuer

	 {4.0459323, 'part-of-speech tagging'}
	 {4.0459323, 'speech recognition'}
	 {4.0459323, 'part-of-speech tagging'}
Moses, Michele S. : 
	 {3.503166, 'part-of-speech tagging'}
	 {'Cognitive science of religion', 3.503166}
Potter, Jason Timothy : 
	 {3.3790565, 'part-of-speech tagging'}
	 {'Cognitive science of religion', 3.3790565}
Liston, Dan : 
	 {3.3351905, 'part-of-speech tagging'}
	 {'Cognitive science of religion', 3.3351905}
Bredeson, Garrett Zantow : 
	 {3.3304708, 'part-of-speech tagging'}
	 {'Cognitive science of religion', 3.3304708}
Savage, Scott James : 
	 {3.2243118, 'part-of-speech tagging'}
	 {'Cognitive science of religion', 3.2243118}
Zhang, Shuang : 
	 {3.1771724, 'part-of-speech tagging'}
	 {'Cognitive science of religion', 3.1771724}
Flores, Nicholas E : 
	 {3.09827, 'part-of-speech tagging'}
	 {'Cognitive science of religion', 3.09827}
Boninger, Faith : 
	 {3.0801952, 'part-of-speech tagging'}
	 {'Cognitive science of religion', 3.0801952}
Wilson, Terri S. : 
	 {3

	 {1.4930458, 'part-of-speech tagging'}
	 {1.4930458, 'Cognitive science of religion'}
Stutzer, Michael "Iron Mike" : 
	 {1.4930458, 'part-of-speech tagging'}
	 {1.4930458, 'Cognitive science of religion'}
Bernstein, Asaf : 
	 {1.4930458, 'part-of-speech tagging'}
	 {1.4930458, 'Cognitive science of religion'}
Bumbaca, Rico : 
	 {1.4930458, 'part-of-speech tagging'}
	 {1.4930458, 'Cognitive science of religion'}
	 {'Marketing decision support system', 5.9472594}
	 {'Marketing decision support system', 5.9472594}
Stephan, Andrew Perry : 
	 {1.4930458, 'part-of-speech tagging'}
	 {1.4930458, 'Cognitive science of religion'}
Gallagher, Emily Anne : 
	 {1.4930458, 'part-of-speech tagging'}
	 {1.4930458, 'Cognitive science of religion'}
Lacerenza, Christina Noelle : 
	 {1.4930458, 'part-of-speech tagging'}
	 {1.4930458, 'Cognitive science of religion'}
White, Terrenda Corisa : 
	 {1.4743667, 'part-of-speech tagging'}
	 {1.4743667, 'Cognitive science of religion'}
Rickels, David A. : 
	 {1.4

	 {'cognitive anthropology', 2.815288}
Fluri, Jennifer L : 
	 {'cognitive geography', 7.043706}
	 {'cognitive geography', 7.043706}
Oakes, Timothy S : 
	 {'cognitive geography', 6.7041745}
	 {'cognitive geography', 6.7041745}
Blanken, Peter David : 
	 {'cognitive geography', 6.1457725}
	 {'sentence boundary disambiguation', 5.064203}
	 {'cognitive geography', 6.1457725}
	 {'sentence boundary disambiguation', 5.064203}
Bryan, Joe : 
	 {'cognitive geography', 6.1457725}
	 {'cognitive geography', 6.1457725}
Veblen, Thomas T : 
	 {'cognitive geography', 6.1457725}
	 {'cognitive geography', 6.1457725}
Lehr, Rachel : 
	 {'cognitive geography', 6.0515337}
	 {'cognitive geography', 6.0515337}
Kelly, Sarah : 
	 {'cognitive geography', 6.0515337}
	 {'cognitive geography', 6.0515337}
Yeh, Emily T. : 
	 {'cognitive geography', 6.0515337}
	 {2.5892448, 'planning expert system'}
	 {'cognitive geography', 6.0515337}
Hickcox, Abby : 
	 {'cognitive geography', 4.9409194}
	 {'cognitive geography', 4.940